In [1]:
# YOLOv8 Training with COCO8 Dataset
# Save this as: notebooks/yolo_training.ipynb

import os
from pathlib import Path
import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch


In [ ]:
from ultralytics import YOLO
import torch
import json
from datetime import datetime
import os

# ==== settings ====
now_str = datetime.now().strftime('%Y%m%d_%H%M%S')
config = {
    "model": "yolov8n.pt",
    "data": "/Users/Kota/blended/Team3AmazonProject/data/furniture.v2-release.yolov8/data.yaml",
    "epochs": 50,
    "imgsz": 320,
    "batch": 32,
    "name": f"furniture_yolov8n_{now_str}",
    "project": "furniture_project_3",
    "exist_ok": True,
    "device": "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu",
    "lr0": 0.01,
    "lrf": 0.01,
    "momentum": 0.937,
    "weight_decay": 0.0005,
    "warmup_epochs": 3.0,
    "warmup_momentum": 0.8,
    "warmup_bias_lr": 0.1,
    "box": 7.5,
    "cls": 0.5,
    "dfl": 1.5,
    "degrees": 0.0,
    "translate": 0.1,
    "scale": 0.5,
    "shear": 0.0,
    "perspective": 0.0,
    "flipud": 0.0,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.0,
    "copy_paste": 0.0,
    "patience": 50,
    "workers": 8,
    "save": True,
    "save_period": -1,
    "cache": False,
    "close_mosaic": 0,
    "resume": False,
    "amp": True,
    "pretrained": True
}

# ==== Train ====
model = YOLO(config["model"])
results = model.train(**config)

# ==== Save ====
save_dir = model.trainer.save_dir  
config_path = os.path.join(save_dir, "config.json")
with open(config_path, "w") as f:
    json.dump(config, f, indent=4)
print(f"✅ 訓練設定を保存しました: {config_path}")

✅ 訓練設定を保存しました: ./furniture_project_3/furniture_yolov8n_0805_204806config_20250805_204806.json
New https://pypi.org/project/ultralytics/8.3.174 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.170 🚀 Python-3.13.5 torch-2.7.1 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/Kota/blended/Team3AmazonProject/data/furniture.v2-release.yolov8/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=trai

train: Scanning /Users/Kota/blended/Team3AmazonProject/data/furniture.v2-release.yolov8/train/labels.cache... 454 images, 0 backgrounds, 0 corrupt: 100%|██████████| 454/454 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 278.0±76.8 MB/s, size: 69.5 KB)



val: Scanning /Users/Kota/blended/Team3AmazonProject/data/furniture.v2-release.yolov8/valid/labels.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

Plotting labels to furniture_project_3/furniture_yolov8n_0805_204806/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to furniture_project_3/furniture_yolov8n_0805_204806
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



       1/50      2.29G     0.7404      2.887      1.091         67        320:  60%|██████    | 9/15 [00:41<00:27,  4.61s/it]


KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
from pathlib import Path

# Input and output directories
input_dir = Path("/Users/Kota/blended/Team3AmazonProject/data/temp/original")
output_dir = Path("/Users/Kota/blended/Team3AmazonProject/data/temp/cropped")
output_dir.mkdir(parents=True, exist_ok=True)

# Load YOLOv8 segmentation model
model = YOLO("yolov8x-seg.pt")

# Supported image extensions
image_exts = [".jpg", ".jpeg", ".png"]

# Iterate over all images in the input directory
for img_path in input_dir.glob("*"):
    if img_path.suffix.lower() not in image_exts:
        continue

    # Read the image
    original = cv2.imread(str(img_path))
    if original is None:
        print(f"⚠️ Failed to read: {img_path}")
        continue

    height, width = original.shape[:2]

    # Run inference without resizing
    results = model(source=str(img_path), imgsz=(width, height))[0]

    # Skip if no masks detected
    if results.masks is None or len(results.masks.data) == 0:
        print(f"❌ No mask detected: {img_path.name}")
        continue

    # Use only the first detected object
    mask = results.masks.data[0].cpu().numpy().astype(np.uint8)

    # Resize the mask to match the original image size
    mask_resized = cv2.resize(mask, (width, height), interpolation=cv2.INTER_NEAREST)

    # Apply the mask to the entire image (black background outside object)
    mask_3c = np.stack([mask_resized] * 3, axis=-1)
    masked_img = original * mask_3c  # Same shape as original

    # Save masked image (same size as original)
    base_name = img_path.stem
    save_path = output_dir / f"{base_name}_cropped.png"
    cv2.imwrite(str(save_path), masked_img)
    print(f"✅ Saved: {save_path}")